In [407]:
import pandas as pd
import numpy as np
import datetime 
import re

In [408]:
fr.iloc[ind[0]+1,1]

2013

In [409]:
# fr.head(20)

In [410]:
res=[[]]
gauge_list = []
nm_list = []
dicts = {}

fr = pd.read_excel('Сев.Двина.xlsx')
ind=fr.loc[fr.iloc[:,0] == "Код поста"].index
fy=int(fr.iloc[ind[0]+1,1])#первый год
for j in ind:#итерация по отдельным таблицам
    nm=[fr.iloc[j,1]]#код поста
    nm_list.append(nm)
    y=int(fr.iloc[j+1,1])#год
    gauge = [fr.iloc[j+2,1]]
    gauge_list.append(gauge)
    
    fd=datetime.datetime(y, 1, 1)#начало года
    ld=datetime.datetime(y+1, 1, 1)#конец года, не включая
    listOfDates=[date for date in np.arange(fd,ld,datetime.timedelta(days=1))]#лист с датами в этом году
    yd=fr.iloc[j+5:j+36,1:13]
    if y%4!=0:#не весокосный год, т.е. остаток от делления на 4 не равен 0:
        #ravel делает из 2d 1d строка за строкой, при этом где-то будут пустые ячейки т.к. не все месяцы по 31 день 
        d=np.delete(yd.values.transpose().ravel(),[59,60,61,123,185,278,340])
    else:
        d=np.delete(yd.values.transpose().ravel(),[60,61,123,185,278,340])
    r=pd.DataFrame(d, columns=nm, index=listOfDates)#делаем datarfame, который по сути series
    if fy==y:
        res[-1].append(r)
    #если пошел следующий год, лист с датафреймами из старого кода соеденяется, а новый датафрейм помещается в лист и 
    #становится основой для сбора расходов со следующего года
    else:
        res[-1]=pd.concat(res[-1], axis=1,sort=False)
        res.append([r])
        fy=y
#последний год объеденяется в единый фрейм уже вне тела цикла
res[-1]=pd.concat(res[-1], axis=1,sort=False)
#наконец все в месте соеденяется
res=pd.concat(res, axis=0,sort=False)

In [411]:
res=res.astype(str)#разные типы данных нам не к чему

In [412]:
#нб - не было в 0, а те ячейки где были дополнительные символы разделяем по пробелу и берем первую часть - число
def str_int(x):
    if x=="нб":
        return 0
    else:
        return float(x.replace(",",".").split(" ")[0])

In [413]:
res=res.applymap(str_int)# применяем функцию ко всем элементам фрейма

In [414]:
res

,70801
2014-01-01,NaN
2014-01-02,NaN
2014-01-03,NaN
2014-01-04,NaN
2014-01-05,NaN
...,...
2019-12-27,NaN
2019-12-28,NaN
2019-12-29,NaN
2019-12-30,NaN


In [415]:
df = res.copy()
df['date'] = df.index

In [416]:
df

,70801,date
2014-01-01,NaN,2014-01-01
2014-01-02,NaN,2014-01-02
2014-01-03,NaN,2014-01-03
2014-01-04,NaN,2014-01-04
2014-01-05,NaN,2014-01-05
...,...,...
2019-12-27,NaN,2019-12-27
2019-12-28,NaN,2019-12-28
2019-12-29,NaN,2019-12-29
2019-12-30,NaN,2019-12-30


In [417]:
df.reset_index(drop=True, inplace=True)
df['date'] = pd.to_datetime(df['date']).apply(lambda x: datetime.datetime.strftime(x, '%#d %#m %Y'))
# df.columns = ['Q', 'date']
df=df.astype(str)
df

,70801,date
0,nan,1 1 2014
1,nan,2 1 2014
2,nan,3 1 2014
3,nan,4 1 2014
4,nan,5 1 2014
...,...,...
2186,nan,27 12 2019
2187,nan,28 12 2019
2188,nan,29 12 2019
2189,nan,30 12 2019


In [418]:
df2 = pd.DataFrame()
for i in df.columns:
    df2[i] = df["date"] + ' ' + df[i]
df2 = df2.drop(columns=['date'])

In [419]:
df2

,70801
0,1 1 2014 nan
1,2 1 2014 nan
2,3 1 2014 nan
3,4 1 2014 nan
4,5 1 2014 nan
...,...
2186,27 12 2019 nan
2187,28 12 2019 nan
2188,29 12 2019 nan
2189,30 12 2019 nan


In [420]:
df2.to_excel('Сев_Двина_для_продления.xlsx')

In [421]:
flat_gauge = [item for sublist in gauge_list for item in sublist]
flat_nm = [item for sublist in nm_list for item in sublist]


In [422]:
list(set(flat_gauge))

['Сев. Двина - р. Северная Двина - с. Усть-Пинега']

In [423]:
list(set(flat_nm))

[70801]